In [1]:
import asyncio
import pandas as pd
import requests
import gender_guesser.detector as gender
import csv
import requests
import os 
import shutil
import time
import re

from tabulate import tabulate
from bs4 import BeautifulSoup
from pathlib import Path
from selenium import webdriver

In [3]:
data = pd.read_excel('/Users/sara/Desktop/BanRep/Pyhton mujeres/Url-autores.xlsx')

### Scraping

In [4]:
for index, row in data.iterrows():
    
    # Startup the webdriver
    driver = webdriver.Chrome()
    url = row['Url']
    c = row['País']
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    try:
        # Extraer investigadores
        div_members = soup.find('div', {'id' : 'members'}) #Encuetro el elemnto div cuyo id es 'members'
        autores = div_members.find('ol') #Encuentro la lista organizada de autores

        a_tag = autores.find_all('a') #Encuentro todos los enlances de los autores
        autor = []

        for tag in a_tag:
          text = tag.get_text().strip() #Para cada autor de la lista, se extrae el texto del enlace
          autor.append(text) #Concateno los nombres

        autor_df = pd.DataFrame(autor, columns=['Nombre']) #Creamos un dataframe de los nombres de los autores

        # Ajustes
        autor_df[['Apellido', 'Nombre ajust']] = autor_df['Nombre'].str.split(', ', expand =True) #Separamos nombre y aprellido

        autor_df[['Primer nombre', 'espacio', 'Segundo']] = autor_df['Nombre ajust'].str.partition(' ') #Separamos nombre por primer espacio
        autor_df = autor_df.drop(['Nombre', 'Nombre ajust', 'espacio'], axis =1) #elimino columnas que no me sirven

        # Identificar genero
        d = gender.Detector()

        # para esto cargo el paquete gender: !pip install gender-guesser
        genero = []
        #Loop para crear identificar el género de cada nombre
        for nombre, row in autor_df.iterrows(): #puedo definir el objeto a iterar antes de definirlo en si mismo (html)
          nombre = row['Primer nombre'] #Ahora sí defino html
          response =  d.get_gender(nombre)
          genero.append(response)

        genero = pd.DataFrame(genero, columns = ['Genero'])
        autor_df = autor_df.join(genero)

        # Guardar data frame
        autor_df['country'] = c
        df_name = c + "_df"
        globals()[df_name] = autor_df.copy()
        
    except:
        pass

In [5]:
# Append all dataframes 
all_data = pd.DataFrame()

for index, row in data.iterrows():
    c = row['País']
    try:
        df_c = globals()[c + '_df']
        all_data = all_data.append(df_c)
    except:
        pass

/var/folders/_p/k0b6_2y92695rmb6_whl6j640000gp/T/ipykernel_7502/4017664693.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(df_c)
/var/folders/_p/k0b6_2y92695rmb6_whl6j640000gp/T/ipykernel_7502/4017664693.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(df_c)
/var/folders/_p/k0b6_2y92695rmb6_whl6j640000gp/T/ipykernel_7502/4017664693.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(df_c)
/var/folders/_p/k0b6_2y92695rmb6_whl6j640000gp/T/ipykernel_7502/4017664693.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(df_c)
/var

In [6]:
all_data

,Apellido,Primer nombre,Segundo,Genero,country
0,Aguirre,Horacio,A.,male,Argentina
1,Balzarotti,Veronica,,female,Argentina
2,Basco,Emiliano,,male,Argentina
3,Bastourre,Diego,Alberto,male,Argentina
4,Bebczuk,Ricardo,N.,male,Argentina
...,...,...,...,...,...
10,Pena,Alejandro,,male,Uruguay
11,Picardo,Pablo,,male,Uruguay
12,Ponce,Jorge,,male,Uruguay
13,Rodriguez,Analia,,unknown,Uruguay


### Estadisticas finales

In [35]:
# Dummy mujer 
fem = ['female', 'mostly_female']
all_data['fem'] = all_data['Genero'].isin(fem).astype(int)

# Colapsar datos
df_sum = all_data.groupby('country')['fem'].agg(['count', 'mean']).reset_index()
df_sum = df_sum.rename(columns={'count': 'Total', 'mean': '% Mujeres'})

United States = 

In [36]:
df_sum

,country,Total,% Mujeres
0,Argentina,23,0.173913
1,Atlanta,33,0.212121
2,Australia,31,0.354839
3,Austria,38,0.236842
4,Bangladesh,10,0.100000
...,...,...,...
67,Ukraine,10,0.100000
68,United Arab Emirates,4,0.250000
69,United Kingdom,213,0.258216
70,Uruguay,14,0.214286


In [37]:
all_data.to_excel('all_data.xlsx', index=False)
df_sum.to_excel('df_sum.xlsx', index=False)